# 一、python操作postgresql介绍

- 导入psycopg2，并创建一个connection连接：connect()<br />
database：设置连接数据库的名称<br />
user：用户名，可以从cmd命令行中，通过“psql molecules”进入数据库，并输入“select usename from pg_user;”查看用户名<br />
password：密码<br />
port：端口号，默认5432<br />
host：宿主ip

In [1]:
import psycopg2
connection = psycopg2.connect(database='molecules',
                              user='hxw',
                              password='postgresql',
                              port='5432',
                              host='127.0.0.1')
type(connection)

psycopg2.extensions.connection

- 创建一个游标cursor，在内存中开辟一块区域，用于存储数据操作的结果：

In [2]:
cur = connection.cursor()
type(cur)

psycopg2.extensions.cursor

- 执行一条SQL语句测试一下，查看操作用户

In [3]:
cur.execute("select * from current_user;")
reply = cur.fetchall()
reply

[('hxw',)]

- 操作失误了可以删除数据表

In [4]:
query = 'drop table info'
cur.execute(query)

- 写一条postgresql语句，记为query，创建一个数据表，命名为info，并表添加列名，设置数据格式
- 使用游标来执行语句：execute(query)

In [5]:
query = '''create table info(id serial primary key,
                             database text,
                             project text,
                             SMILES text,
                             HA integer,
                             HD integer,
                             RB integer,
                             MW float,
                             LOGP float);'''
cur.execute(query)

In [6]:
connection.commit()

- 读取1000条左右的数据

In [7]:
import pandas as pd
df = pd.read_excel('ippin.xlsx')
df.head(1)

,Name,CAS #,SMILES,human VDss (L/kg),human CL (mL/min/kg),fraction unbound \nin plasma (fu),MRT (h),terminal t1/2 (h),Reference,Comments,Notes,Year of first disclosure,MW,HBA,HBD,TPSA_NO,RotBondCount,moka_ionState7.4,MoKa.LogP,MoKa.LogD7.4
0,α-hANP,85637-73-6,N1[C@H](C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@H](C(...,0.2,25.4,NaN,0.13,0.22,"Nakao, K.; Sugawara, A.; Morii, N.; Sakamoto, ...",α-Human Atrial Natriuretic Peptide. MRT calcul...,NaN,1982,3080.44,84,53,1403.4,75,cationic,-9.0,-9.0


- 获取mol对象列表

In [8]:
from rdkit import Chem
mol_list = [x for x in [Chem.MolFromSmiles(i) for i in df.SMILES] if x]
len(mol_list)

1351

- 创建一个描述符计算对象，将类药五规则设置到计算器中，这部分不明白的可以我看我之前的文章

In [9]:
from rdkit.ML.Descriptors import MoleculeDescriptors
des_list = ['MolWt', 'NumHAcceptors', 'NumHDonors', 'MolLogP', 'NumRotatableBonds']
calculator = MoleculeDescriptors.MolecularDescriptorCalculator(des_list)

- 使用计算器，对每个分子计算所需描述符

In [10]:
feat_list = ['%s'%(str(calculator.CalcDescriptors(mol))) for mol in mol_list]

- 在query中写入一条向数据表info插入数据的语句，并执行该语句

In [11]:
query = "insert into info (MW, HA, HD, LOGP, RB) values %s" % (','.join(feat_list))
cur.execute(query)

- 通过聚合函数count(*)看一下有多少条结果
- 通过游标获取查询结果：fetchall()

In [12]:
query = 'select count(*) from info'
cur.execute(query)
reply = cur.fetchall()
reply

[(1351,)]

- 最后操作完后，记得提交对数据库的修改：commit()

In [13]:
connection.commit()

# 二、自动化导入脚本
对dataframe中的分子计算类药五规则，并自动导入到数据库中。

In [14]:
import os
import psycopg2
import time
import pandas as pd
from rdkit import Chem
from rdkit.ML.Descriptors import MoleculeDescriptors

def import_data(df, smi_col, database, project=None, table='info', renew=False):
    '''Import smiles and corresponding properties into the postgreSQL automatically.
    
    df: the DataFrame containing SMILES of the mols.
    smi_col: the column name of the SMILES.
    database: the PG database that want to modified.
    project: the project that the mols belong to.
    table: the table in the database that will be modified.
    renew: drop the table and create a new one.
    '''
    # connect to database and test for connection
    time1 = time.time()
    connection = psycopg2.connect(database=database,
                                  user='hxw',
                                  password='postgresql',
                                  port='5432',
                                  host='127.0.0.1')
    
    cur = connection.cursor()
    cur.execute("select * from current_user;")
    print('test for connection:', cur.fetchall()[0][0])

    # examine whether the table exist
    query = "select count(*) from pg_class where relname = '%s';"%table
    cur.execute(query)
    exist = cur.fetchall()[0][0]
    
    # renew or create a table
    if not exist or renew:
        if not exist:
            print('"%s" not found in "%s" and will be created'%(table, database))
        if renew:
            print('drop the %s and create a new one'%table)
            query = 'drop table %s'%table
            cur.execute(query)
        query = '''create table %s(id serial primary key,
                                   database text,
                                   project text,
                                   SMILES text,
                                   HA integer,
                                   HD integer,
                                   RB integer,
                                   MW float,
                                   LOGP float);'''%table
        cur.execute(query)

    # calc the descriptors
    des_list = ['MolWt', 'NumHAcceptors', 'NumHDonors', 'MolLogP', 'NumRotatableBonds']
    calculator = MoleculeDescriptors.MolecularDescriptorCalculator(des_list)
    feat_all = []
    create_var = locals()
    for mol_smiles in df[smi_col]:
        mol = Chem.MolFromSmiles(mol_smiles)
        if not mol:
            continue
        feat_mol = []
        for item in calculator.CalcDescriptors(mol):
            if isinstance(item, float) or isinstance(item, int):
                item = str(round(item, 3))
            feat_mol.append(item)
        feat_mol.append("'%s'"%mol_smiles)
        feat_all.append("(%s)"%(','.join(feat_mol)))

    # import the data into database and set the project
    query = "insert into %s (MW, HA, HD, LOGP, RB, SMILES) values %s" % (table, ','.join(feat_all))
    cur.execute(query)
    if project:
        query = "update %s set project = '%s'"%(table, project)
        cur.execute(query)
    connection.commit()
    
    time2 = time.time()
    print('using time:', time2-time1)

df = pd.read_excel('ippin.xlsx')
import_data(df, 'SMILES', 'molecules', 'ippih', 'info', True)

test for connection: hxw
drop the info and create a new one
using time: 2.6931540966033936
